In [1]:
import pandas as pd
import json

In [2]:
# прочитаем train_2_col.csv
train_4_col_2 = pd.read_csv('train_4_col_2.csv', sep='\t')
train_4_col_2_small = train_4_col_2.head(100)
train_4_col_2_small.head()

,Unnamed: 0,id,category,subcategory
0,0,3edeb34cf93f490ff760af85,9,914
1,1,c98febd50dad3cc0ffc86085,22,2202
2,2,ade01e13912a46a99134cc75,22,2204
3,3,ab3e6941c11304c1519aef75,22,2209
4,5,252452a91c944a22c276d995,22,2204


In [3]:
# прочитаем train_4_col_3 .csv
train_4_col_3 = pd.read_csv('train_4_col_3.csv', sep='\t')
train_4_col_3_small = train_4_col_3.head(100)
train_4_col_3_small.head()

,Unnamed: 0,id,fields
0,0,3edeb34cf93f490ff760af85,"[{'field': {'name': 'Женский гардероб', 'id': ..."
1,1,c98febd50dad3cc0ffc86085,"[{'field': {'name': 'Детский гардероб', 'id': ..."
2,2,ade01e13912a46a99134cc75,"[{'field': {'name': 'Детский гардероб', 'id': ..."
3,3,ab3e6941c11304c1519aef75,"[{'field': {'name': 'Детский гардероб', 'id': ..."
4,5,252452a91c944a22c276d995,"[{'field': {'name': 'Детский гардероб', 'id': ..."


In [4]:
# прочитаем train_4_col.csv
train_4_col = pd.read_csv('train_4_col.csv', sep='\t')
train_4_col_small = train_4_col.head(100)
train_4_col_small.head()

,Unnamed: 0,id,name,description,price
0,0,3edeb34cf93f490ff760af85,Сумка DG,NaN,199900
1,1,c98febd50dad3cc0ffc86085,Комплект,8-12 лет,35000
2,2,ade01e13912a46a99134cc75,Пальтишко демисезонное,"На девочку 1,5 г,состояние хорошее",30000
3,3,ab3e6941c11304c1519aef75,Attipas,"Размер-135mm, euro-22.5.Прочная, мягкая, не ск...",80000
4,5,252452a91c944a22c276d995,Жилет теплый,NaN,50000


#### Объединим маленькие датафреймы

#### Внимание!!! чтобы объединились полные датафреймы - надо убрать окончание "_small" в ячейке ниже. Далее будет использовано название train. После объединения полных датафреймов это название менять не надо.

In [5]:
train = train_4_col_3_small.merge(train_4_col_2_small, on='id', how='left').merge(train_4_col_small, on='id', how='left')
train.head(2).T

,0,1
Unnamed: 0_x,0,1
id,3edeb34cf93f490ff760af85,c98febd50dad3cc0ffc86085
fields,"[{'field': {'name': 'Женский гардероб', 'id': ...","[{'field': {'name': 'Детский гардероб', 'id': ..."
Unnamed: 0_y,0,1
category,9,22
subcategory,914,2202
Unnamed: 0,0,1
name,Сумка DG,Комплект
description,NaN,8-12 лет
price,199900,35000


In [6]:
# Функция, которая извлекает значений ключей из поля "fields"
def valextractor(field, key1=0, key2='name'):
    value = eval(field[1:-1])[key1]['field'][key2] 
    return value

In [7]:
# проверяем функцию. работает.
field = train.iloc[0,2]
valextractor(field, key1=0,key2='name')

'Женский гардероб'

In [8]:
# функция, которая доавляет новые категории
def getcat(df):
    df_ = df.copy()
    df_['fd_name1'] = df_['fields'].map(lambda x: (valextractor(x, key1=0,key2='name')))
    df_['fd_name2'] = df_['fields'].map(lambda x: (valextractor(x, key1=1,key2='name')))
    df_['fd_id1'] = df_['fields'].map(lambda x: (valextractor(x, key1=0,key2='id')))
    df_['fd_id2'] = df_['fields'].map(lambda x: (valextractor(x, key1=1,key2='id')))
    df_['fd_slug1'] = df_['fields'].map(lambda x: (valextractor(x, key1=0,key2='slug')))
    df_['fd_slug2'] = df_['fields'].map(lambda x: (valextractor(x, key1=1,key2='slug')))
    df_['fd_slug_id1'] = df_['fields'].map(lambda x: (valextractor(x, key1=0,key2='slug_id')))
    df_['fd_slug_id2'] = df_['fields'].map(lambda x: (valextractor(x, key1=1,key2='slug_id')))
    return df_

In [9]:
# применим функцию
train = train.pipe(getcat)

In [10]:
# удалим задублированные столбцы
train.drop(columns=['fd_id1','fd_id2','Unnamed: 0_y','Unnamed: 0_x','fields' ],axis=1, inplace=True)
# и переименуем 
train.rename(columns = {'Unnamed: 0':'some_column'}, inplace = True) 
# посмотрим что получилось
train.head(50).T

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
id,3edeb34cf93f490ff760af85,c98febd50dad3cc0ffc86085,ade01e13912a46a99134cc75,ab3e6941c11304c1519aef75,252452a91c944a22c276d995,364f7659a4e9ba6c53e35495,c3d87065e4dbc080f400bb85,22e6a518d53f490f6ce7fd85,c6bd550b7f1304c1f7fda485,117b885a3cf9554042ae2285,...,3438949c05e3b470c984ea95,266fc4d1395ccdeb3428fe85,62c792b5c1141fee419acc95,3aec3024bc0e788cdb046795,62df3b3861f95540e9834385,3a80d58fd82d9e58ce4aed85,30b215c9295ccdeb1abe2195,060658c789d3f35d14320d75,4a23d59792944a225801d395,3105314ef46c1e2633b58195
category,9,22,22,22,22,9,15,4,3,2,...,3,3,22,14,16,22,3,9,22,100
subcategory,914,2202,2204,2209,2204,902,1508,403,301,208,...,317,314,2209,1403,1608,2209,314,902,2211,10001
some_column,0,1,2,3,5,6,7,8,10,11,...,59,60,61,62,64,65,66,68,69,70
name,Сумка DG,Комплект,Пальтишко демисезонное,Attipas,Жилет теплый,"Босоножки, сабо",Жесткий диск для ноутбука Seagate 320gb,Ковер 324 на 240,Кенгуру,Машинка автомат,...,Электронная качель,Покебол со случайной минифигуркой покемона внутри,Ботинки зимние,Духи Lancome la vie est belle,Набор для маникюра,Босоножки для танцев,Коврик музыкальный,Туфли р .36,Нарядное платье,Детское питание
description,NaN,8-12 лет,"На девочку 1,5 г,состояние хорошее","Размер-135mm, euro-22.5.Прочная, мягкая, не ск...",NaN,"Новые, текстильные босоножки, 37 размер в разм...",Seagate Модель St320lt020 HDD для ноутбуков ...,"Состояние отличное,без дефектов","Продам кенгуру 350 , ванночка с горкой 250",NaN,...,"Электронная качель Baby Care Riva, удобная, ко...",Размер покебола: 7 см.,"Зимние, размер 20-21,( так написано на подошве...","Edp! Оригинал! Объём 30 мл, осталось половина....",На моем профиле моного интересных вещей,Для танцев стелька 22см,"Для возраста 3+, несколько режимов работы","Туфли, кожа натуральная. Подходят для школы. О...","Красивое нарядное платье, размер 104, внутри т...",Продам 2 пачки новых каш с грушей и бананом. Ц...
price,199900,35000,30000,80000,50000,30000,150000,200000,35000,350000,...,260000,30000,80000,90000,25000,50000,30000,100000,40000,10000
fd_name1,Женский гардероб,Детский гардероб,Детский гардероб,Детский гардероб,Детский гардероб,Женский гардероб,Компьютерная техника,Для дома и дачи,Детские товары,Бытовая техника,...,Детские товары,Детские товары,Детский гардероб,Красота и здоровье,Услуги,Детский гардероб,Детские товары,Женский гардероб,Детский гардероб,Прочее
fd_name2,Аксессуары,Аксессуары,Верхняя одежда,Обувь,Верхняя одежда,Обувь,Комплектующие и запчасти,Текстиль и ковры,Другое,Стиральные машины,...,Обустройство детской,Игрушки и игры,Обувь,Парфюмерия,Красота и здоровье,Обувь,Игрушки и игры,Обувь,Платья и юбки,Другое
fd_slug1,zhenskaya-odezhda,detskaya-odezhda,detskaya-odezhda,detskaya-odezhda,detskaya-odezhda,zhenskaya-odezhda,kompyutery,dom-dacha,detskie,bytovaya-tekhnika,...,detskie,detskie,detskaya-odezhda,krasota-i-zdorove,uslugi,detskaya-odezhda,detskie,zhenskaya-odezhda,detskaya-odezhda,prochee


In [11]:
train.to_csv('data/train.csv')

In [12]:
full_columns_list = list(train) # полцчим список столбцов 
num_cat_columns = ['category','subcategory'] # список числовых категориальных столбцов
#let_cat_columns = full_columns_list[-1] # список буквенных категориальных столбцов
target = 'price'

#### Далее идут наработки по созданию числовых лэйблов и One Hote Encoding из них. Хотя пока еще рановато. Надо еще tf-idf сделать

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer

In [ ]:
lbl_enc = LabelEncoder()
cat_data = train_4_col_3_small_new['fd_slug_id1']
cat_data_encoded = lbl_enc.fit_transform(cat_data)
cat_data_encoded